HW2 part1

Задача: 
1) Загрузите датасет по ценам на жилье Airbnb, доступный на kaggle.com: https://www.kaggle.com/dgomonov/new-york-cityairbnb-open-data
2) реализуйте скрипт mapper.py и reducer.py для расчета, и с помощью MapReduce рассчитайте среднее значение и дисперсию по признаку ’price’ (возможно потребуются ассимптотические формулы для дисперсии);
3) Проверьте правильность подсчета статистики методом mapreduce в сравнении со hive.

In [1]:
!pip install opendatasets

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
!pip install pandas

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
import opendatasets as od
import pandas as pd
import time
from pathlib import Path

In [4]:
import json
from functools import reduce

In [5]:
dataset_path = Path('new-york-city-airbnb-open-data', 'AB_NYC_2019.csv')
if not dataset_path.is_file():
    od.download('https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data')

In [6]:
%pwd new-york-city-airbnb-open-data/  # Посмотреть текущий рабочий каталог

'/opt/workspace'

In [7]:
!ls new-york-city-airbnb-open-data/ # Показать все файлы в текущем каталоге

 AB_NYC_2019.csv   HW2_MapReduce.ipynb	 New_York_City_.png  'Untitled Folder'


In [8]:
df = pd.read_csv(dataset_path) # импорт файла с разделителем по-умолчанию

In [9]:
df_chunks = pd.read_csv(dataset_path, chunksize=1)

In [10]:
def mapper(chunk):
    return (int(chunk['price']), )

def reducer(score_data1, score_data2):
    if score_data1 is None and score_data2 is None:
      return None
    elif score_data1 is None:
      return score_data2
    elif score_data2 is None:
      return score_data1
    else:
      scores = []
      if len(score_data1) == 1:
        n, mean, M2 = 0, 0.0, 0
        scores.append(score_data1[0])
      else:
        n, mean, M2 = score_data1
      scores.append(score_data2[0])
      for score in scores:
        n += 1
        delta = score - mean
        mean += delta / n
        M2 += delta * (score - mean)
      return n, mean, M2

In [11]:
# Параллельная обработка с использованием map и reduce:

In [12]:
%%time
n, mean, M2 = reduce(reducer, map(mapper, df_chunks))
print(mean, (M2 / n) ** (1/2))
print(n, mean, M2)

152.72068718682823 240.15171391941672
48895 152.72068718682823 2819913790.4249
CPU times: user 1min 37s, sys: 942 ms, total: 1min 38s
Wall time: 1min 38s


In [13]:
# Вычислим среднее значение набора значений, переданных в функцию с помощью функции mean() ps: cреднее значение — это число, выражающее центральное или типичное значение в наборе данных, в частности модальное значение, медиана или (чаще всего) среднее значение, которое рассчитывается путём деления суммы значений в наборе на их количество

In [14]:
df['price'].mean()

152.7206871868289

In [15]:
# Вычислим дисперсию(сумма квадратов разностей между всеми числами и средним значением) столбца "price" в датафрейме df:

In [16]:
df['price'].var()

57674.02524696099

In [17]:
# Пример вычисления дисперсии столбца "price" по группам "neighbourhood_group" в датафрейме df:

In [18]:
df.groupby('neighbourhood_group')['price'].var()

neighbourhood_group
Bronx            11386.885081
Brooklyn         34921.719135
Manhattan        84904.159185
Queens           27923.130227
Staten Island    77073.088342
Name: price, dtype: float64

In [19]:
# Вычислим стандартное отклонение (это квадратный корень из дисперсии - мера того, насколько данные отличаются от среднего значения):

In [20]:
df['price'].std( )

240.15416974718758

In [21]:
# Выведем на экран содержимое датафрейма:

In [22]:
df.info

<bound method DataFrame.info of              id                                               name   host_id  \
0          2539                 Clean & quiet apt home by the park      2787   
1          2595                              Skylit Midtown Castle      2845   
2          3647                THE VILLAGE OF HARLEM....NEW YORK !      4632   
3          3831                    Cozy Entire Floor of Brownstone      4869   
4          5022   Entire Apt: Spacious Studio/Loft by central park      7192   
...         ...                                                ...       ...   
48890  36484665    Charming one bedroom - newly renovated rowhouse   8232441   
48891  36485057      Affordable room in Bushwick/East Williamsburg   6570630   
48892  36485431            Sunny Studio at Historical Neighborhood  23492952   
48893  36485609               43rd St. Time Square-cozy single bed  30985759   
48894  36487245  Trendy duplex in the very heart of Hell's Kitchen  68119814   

       

In [23]:
df.info(memory_usage=True) # посмотреть ресурсы памяти

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

In [24]:
df['price'] # Выведем значения только столбца price:

0        149
1        225
2        150
3         89
4         80
        ... 
48890     70
48891     40
48892    115
48893     55
48894     90
Name: price, Length: 48895, dtype: int64

In [ ]:
for index, row in df.iterrows():
    print(row[0:20]) # Выведем значения всех строчек с помощью метода iterrows():

In [ ]:
for index, row in df.iterrows():
    print(row['price']) # Отобразим все элементы столбца price: